In [1]:
import os
import pandas as pd
import plotly.express as px
from google.cloud import bigquery
from decimal import Decimal

pd.set_option('max_colwidth', 70)

In [2]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "../../cloud_key.json"

In [3]:
FROM_BLOCK_TIMESTAMP = "'2020-11-02 00:00:00'"
TO_BLOCK_TIMESTAMP = "'2020-11-20 23:59:59'" 

MY_TIMEOUT = 300

client = bigquery.Client()

## Uniswap Events

In [4]:
query = """SELECT * FROM `blockchain-etl.ethereum_uniswap.UniswapV2Pair_event_Swap` WHERE contract_address = '0xa478c2975ab1ea89e8196811f51a7b7ade33eb11' AND block_number >= 11283273""".format(
    from_block_ts=FROM_BLOCK_TIMESTAMP, 
    to_block_ts=TO_BLOCK_TIMESTAMP)

query_job = client.query(query)
iterator = query_job.result(timeout=MY_TIMEOUT)

In [5]:
rows = list(iterator)
uniswap = pd.DataFrame(data=[list(x.values()) for x in rows], columns=list(rows[0].keys()))
uniswap = uniswap.sort_values(['block_number','log_index']).reset_index(drop=True)
uniswap.drop_duplicates(subset='transaction_hash', keep=False, inplace=True)
uniswap.rename(columns=
               {"amount0In":"dai_amount_in",
                "amount1In":"eth_amount_in",
                "amount0Out":"dai_amount_out",
                "amount1Out":"eth_amount_out"},
                inplace=True)

In [6]:
uniswap['dai_amount_in'] = uniswap['dai_amount_in'].apply(Decimal)
uniswap['dai_amount_out'] = uniswap['dai_amount_out'].apply(Decimal)
uniswap['eth_amount_in'] = uniswap['eth_amount_in'].apply(Decimal)
uniswap['eth_amount_out'] = uniswap['eth_amount_out'].apply(Decimal)

In [7]:
def calc_dai_price(row):
    if (row.dai_amount_out != 0) & (row.eth_amount_in != 0):
        return row.dai_amount_out / row.eth_amount_in
    else:
        return row.dai_amount_in / row.eth_amount_out

def calc_eth_price(row):
    if (row.eth_amount_out !=0) & (row.dai_amount_in != 0):
        return row.eth_amount_out / row.dai_amount_in
    else:
        return row.eth_amount_in / row.dai_amount_out

In [8]:
#uniswap['dai_price'] = uniswap.apply(lambda row: calc_dai_price(row), axis=1)
#uniswap['eth_price'] = uniswap.apply(lambda row: calc_eth_price(row), axis=1)

uniswap['dai_amount_out'][uniswap['dai_amount_out'] != 0] / uniswap['eth_amount_in'][uniswap['eth_amount_in'] != 0] 
uniswap['dai_price'] = uniswap['dai_amount_in'][uniswap['dai_amount_in'] != 0] / uniswap['eth_amount_out'][uniswap['eth_amount_out'] != 0] 
uniswap['eth_price'] = uniswap['eth_amount_out'][uniswap['eth_amount_out'] != 0] / uniswap['dai_amount_in'][uniswap['dai_amount_in'] != 0]
uniswap['eth_price'] = uniswap['eth_amount_in'][uniswap['eth_amount_in'] != 0] / uniswap['dai_amount_out'][uniswap['dai_amount_out'] != 0]
uniswap

,block_timestamp,block_number,transaction_hash,log_index,contract_address,sender,dai_amount_in,eth_amount_in,dai_amount_out,eth_amount_out,to,dai_price,eth_price
0,2020-11-18 17:28:57+00:00,11283273,0x033eba188abdff3fd4b5de6c292c427727b729933c4c41bc48b26dfd43ee46b4,39,0xa478c2975ab1ea89e8196811f51a7b7ade33eb11,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,0,178373339036740659,84000000000000000000,0,0xc47dec7ffde829043b91e904fdaf1e048bdd482c,NaN,0.00212349213138976975
1,2020-11-18 17:28:57+00:00,11283273,0x2024c774e832ac2b059c5bacc3f3736d8e50f5997658daed17ab4bb7dcad5c19,61,0xa478c2975ab1ea89e8196811f51a7b7ade33eb11,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,897587599446926563429,0,0,1894585560969778077,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,473.7646153006043315626955134,NaN
2,2020-11-18 17:28:57+00:00,11283273,0x1c90931720f48b5cd3cea5a1a6c64423dd812a5f3ca79a0875b6b59133708d72,261,0xa478c2975ab1ea89e8196811f51a7b7ade33eb11,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,899033824865797806287,0,0,1897603438171453174,0x1107b6081231d7f256269ad014bf92e041cb08df,473.7732904468778011791623465,NaN
3,2020-11-18 17:28:58+00:00,11283274,0x5a89cc2850fabbda425bd025966be5d470ed1cdb3f4a56bbb34761df53f16d0b,177,0xa478c2975ab1ea89e8196811f51a7b7ade33eb11,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,0,1000000000000000000,470936972491528376645,0,0x427e5697747b64097864bc681d59ae4aab3dccb6,NaN,0.002123426399735452643146684404
4,2020-11-18 17:29:08+00:00,11283275,0x729e1164d74690afc651a874f5d368152bd756522200e69d7f985becd9af0e83,140,0xa478c2975ab1ea89e8196811f51a7b7ade33eb11,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,0,226853230566684541,106832942022602339465,0,0xf00e80f0de9aea0b33aa229a4014572777e422ee,NaN,0.002123438953115134215329270977
...,...,...,...,...,...,...,...,...,...,...,...,...,...
631153,2021-05-24 20:56:18+00:00,12499334,0xb2e2cda3cb14b8b0fdbdedb9e5ee2b901f767e20c85b17ffa814e8945f7c1eb6,166,0xa478c2975ab1ea89e8196811f51a7b7ade33eb11,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,0,3531749261365351777,9193235653459291292058,0,0x3ddd02178297e2f55d5cd3f503586bf6165a2d7b,NaN,0.0003841682509287578081256946180
631154,2021-05-24 20:58:15+00:00,12499340,0xac810ff85e9c49fb91d29eaec00202926eafacdbb93ef7eafa36700a777f26ec,96,0xa478c2975ab1ea89e8196811f51a7b7ade33eb11,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,213009497135342316140,0,0,81357628687627414,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,2618.187139563668989879808248,NaN
631155,2021-05-24 20:58:18+00:00,12499341,0xc5c230229c3ff088b2bacfa972e16a8c63d87d9a86b9c68f82b77f36c95f9b68,81,0xa478c2975ab1ea89e8196811f51a7b7ade33eb11,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,543867147715777757952,0,0,207723103233038242,0x3328f5f2cecaf00a2443082b657cedeaf70bfaef,2618.231382311045678289352427,NaN
631156,2021-05-24 20:58:26+00:00,12499342,0xf19b00dc0d264d3c854adb6f506f3b15fd007735c86ac1cac6ec521f3c93b0d0,328,0xa478c2975ab1ea89e8196811f51a7b7ade33eb11,0xdef1c0ded9bec7f1a1670819833240f027b25eff,659102442598017921963,0,0,251728986393417772,0xdef1c0ded9bec7f1a1670819833240f027b25eff,2618.301738076089043418215978,NaN


In [9]:
# uniswap['transaction_fee'] = uniswap['gas_used'] * uniswap['gas_price']
# uniswap['eth_used'] = uniswap['eth_amount_in'] * uniswap['transaction_fee']

## Balancer Events

In [10]:
contracts = {
    '0x7afe74ae3c19f070c109a38c286684256adc656c'.lower(): 'ETH/DAI'
}

contracts

{'0x7afe74ae3c19f070c109a38c286684256adc656c': 'ETH/DAI'}

In [11]:
query = """
SELECT 
  events.transaction_hash AS transactionHash,
  events.transaction_index,
  txns.from_address AS transaction_sender, 
  txns.receipt_gas_used AS gas_used,
  txns.gas_price,
  events.address,
  events.data,
  events.topics,
  events.block_timestamp,
  events.block_number AS blockNumber_dec
FROM
  `bigquery-public-data.crypto_ethereum.logs` AS events
INNER JOIN
  `bigquery-public-data.crypto_ethereum.transactions` AS txns
ON
  events.transaction_hash = txns.hash
WHERE TRUE
  AND events.block_timestamp >= {from_block_ts} 
  AND events.block_timestamp <= {to_block_ts}
  AND txns.block_timestamp >= {from_block_ts} --might seem redundant, but because of partitioning this reduces cost
  AND txns.block_timestamp <= {to_block_ts} --might seem redundant, but because of partitioning this reduces cost
  AND txns.to_address IN ('0x3e66b66fd1d0b02fda6c811da9e0547970db2f21')
  AND events.address IN ({contract_list})
""".format(
    from_block_ts=FROM_BLOCK_TIMESTAMP, 
    to_block_ts=TO_BLOCK_TIMESTAMP, 
    contract_list=(','.join(["'{}'".format(k) for k in list(contracts.keys())])))

query_job = client.query(query)
iterator = query_job.result(timeout=MY_TIMEOUT)
rows = list(iterator)
balancer = pd.DataFrame(data=[list(x.values()) for x in rows], columns=list(rows[0].keys()))
balancer['contract'] = balancer['address'].apply(lambda x: contracts[x])

balancer.head(10)

,transactionHash,transaction_index,transaction_sender,gas_used,gas_price,address,data,topics,block_timestamp,blockNumber_dec,contract
0,0x1bef7edabc0d155060807905fc56e6035d3492eb9405740e9a214285a4ca91f1,1,0xc8832ca099980dcaa62c9677cdd8805a94643428,158569,132000000000,0x7afe74ae3c19f070c109a38c286684256adc656c,0x0000000000000000000000000000000000000000000000000000000000000020...,[0x8201aa3f0000000000000000000000000000000000000000000000000000000...,2020-11-19 13:40:58+00:00,11288753,ETH/DAI
1,0xd5b804e3b14b83ec837ca66f41bd4e3db5db5592b86c8d78924a35007640a425,116,0x2c7cc6dd30d1035962d9ddb263cf392152c789c5,276966,24000000000,0x7afe74ae3c19f070c109a38c286684256adc656c,0x0000000000000000000000000000000000000000000000000098990dd14bb66b...,[0x908fb5ee8f16c6bc9bc3690973819f32a4d4b10188134543c88706e0e1d4337...,2020-11-19 21:46:01+00:00,11290998,ETH/DAI
2,0xca6cf74991c9a94deb4c82ac0cb284d56b5880a4b216d3d5eb13f7778913ac36,313,0xdcbde0cfdf9fbbe309a3c4c1c2e8c52cde434a4a,229430,39000000000,0x7afe74ae3c19f070c109a38c286684256adc656c,0x000000000000000000000000000000000000000000000000089aaeb710be0000...,[0x908fb5ee8f16c6bc9bc3690973819f32a4d4b10188134543c88706e0e1d4337...,2020-11-19 20:35:35+00:00,11290671,ETH/DAI
3,0xff4a70e121e150a9c929e0ed46ef83416de86084c28fe686871e18023feb8f5a,46,0x66d0c008ffb4730fa7c5025e1f11f0685ff951c5,152158,32000000000,0x7afe74ae3c19f070c109a38c286684256adc656c,0x0000000000000000000000000000000000000000000000000000000000000020...,[0x8201aa3f0000000000000000000000000000000000000000000000000000000...,2020-11-19 09:36:51+00:00,11287650,ETH/DAI
4,0x1309d5c2e53a26712862b6a3b12d5cabc676d2e577bfbfb97d599e19ff4c8fd3,125,0x9476aed77db00a375dd75cd3b5d6031e4eb3070a,324066,21100000000,0x7afe74ae3c19f070c109a38c286684256adc656c,0x00000000000000000000000000000000000000000000000000e76d16a2692af4...,[0x908fb5ee8f16c6bc9bc3690973819f32a4d4b10188134543c88706e0e1d4337...,2020-11-19 07:10:49+00:00,11286986,ETH/DAI
5,0x0633216cac32dae62070b278780eb4fd18aff399dfdd4eac8cbafe3e4a6a81c4,118,0x2c7cc6dd30d1035962d9ddb263cf392152c789c5,325883,19800000000,0x7afe74ae3c19f070c109a38c286684256adc656c,0x000000000000000000000000000000000000000000000000012e91a3df232fa5...,[0x908fb5ee8f16c6bc9bc3690973819f32a4d4b10188134543c88706e0e1d4337...,2020-11-19 21:38:31+00:00,11290974,ETH/DAI
6,0xda1e1c9d93d6f86a43c3c55709cb6989995dd2610b22a503ef5de67fbc00cc26,143,0x8d549dfa7ea7aba384ec2b98f293e3d3c784bdf9,237759,19800000000,0x7afe74ae3c19f070c109a38c286684256adc656c,0x0000000000000000000000000000000000000000000000000000000000000020...,[0x8201aa3f0000000000000000000000000000000000000000000000000000000...,2020-11-19 21:33:52+00:00,11290954,ETH/DAI
7,0x28a422dc2cfb09d63b0a64cad7623b6bc87eb88353e6851535e5c5b9b781648f,113,0xeb2e5d2c188a8ffabde4bffe91bc0a1991d416bc,255296,50000000000,0x7afe74ae3c19f070c109a38c286684256adc656c,0x0000000000000000000000000000000000000000000000000000000000000020...,[0x8201aa3f0000000000000000000000000000000000000000000000000000000...,2020-11-19 00:28:46+00:00,11285144,ETH/DAI
8,0x2276b0b79e675ebe55bcaa83981f7a46104e2f6a06eb4552ed7b442c967115d5,114,0xc6d9f798ac7d130e51f241f8f3e95961e21e171c,243923,30000000000,0x7afe74ae3c19f070c109a38c286684256adc656c,0x0000000000000000000000000000000000000000000000000000000000000020...,[0x8201aa3f0000000000000000000000000000000000000000000000000000000...,2020-11-19 09:08:35+00:00,11287519,ETH/DAI
9,0xd5b804e3b14b83ec837ca66f41bd4e3db5db5592b86c8d78924a35007640a425,116,0x2c7cc6dd30d1035962d9ddb263cf392152c789c5,276966,24000000000,0x7afe74ae3c19f070c109a38c286684256adc656c,0x0000000000000000000000000000000000000000000000000000000000000020...,[0x7c5e9ea40000000000000000000000000000000000000000000000000000000...,2020-11-19 21:46:01+00:00,11290998,ETH/DAI


In [12]:
sql = """
    select * from `blockchain-etl.ethereum_balancer.view_token_balances_subset`
    where address in ('0x7afe74ae3c19f070c109a38c286684256adc656c')
    and block_number >= 12403257 
    and block_number <= 12448607
    and balance > 0
"""

In [13]:
query_job = client.query(sql)
iterator = query_job.result(timeout=MY_TIMEOUT)
rows = list(iterator)
balances = pd.DataFrame(data=[list(x.values()) for x in rows], columns=list(rows[0].keys()))

In [14]:
unchanged_balance = pd.DataFrame(balances[balances['balance'] == balances['balance'].shift(1)])
unchanged_balance

,token_address,address,block_number,balance
3,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0x7afe74ae3c19f070c109a38c286684256adc656c,12414285,4.593556e+20
5,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0x7afe74ae3c19f070c109a38c286684256adc656c,12415460,4.521752e+20
6,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0x7afe74ae3c19f070c109a38c286684256adc656c,12415465,4.521752e+20
7,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0x7afe74ae3c19f070c109a38c286684256adc656c,12415454,4.521752e+20
10,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0x7afe74ae3c19f070c109a38c286684256adc656c,12433179,1.404694e+21
...,...,...,...,...
90696,0x6b175474e89094c44da98b954eedeac495271d0f,0x7afe74ae3c19f070c109a38c286684256adc656c,12446395,5.432919e+24
90697,0x6b175474e89094c44da98b954eedeac495271d0f,0x7afe74ae3c19f070c109a38c286684256adc656c,12446354,5.432919e+24
90698,0x6b175474e89094c44da98b954eedeac495271d0f,0x7afe74ae3c19f070c109a38c286684256adc656c,12446418,5.432919e+24
90700,0x6b175474e89094c44da98b954eedeac495271d0f,0x7afe74ae3c19f070c109a38c286684256adc656c,12446989,5.394263e+24


In [15]:
uniswap = pd.concat([uniswap[~uniswap['block_number'].isin(balances['block_number'])],
                        uniswap[uniswap['block_number'].isin(unchanged_balance['block_number'])]])
uniswap

,block_timestamp,block_number,transaction_hash,log_index,contract_address,sender,dai_amount_in,eth_amount_in,dai_amount_out,eth_amount_out,to,dai_price,eth_price
0,2020-11-18 17:28:57+00:00,11283273,0x033eba188abdff3fd4b5de6c292c427727b729933c4c41bc48b26dfd43ee46b4,39,0xa478c2975ab1ea89e8196811f51a7b7ade33eb11,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,0,178373339036740659,84000000000000000000,0,0xc47dec7ffde829043b91e904fdaf1e048bdd482c,NaN,0.00212349213138976975
1,2020-11-18 17:28:57+00:00,11283273,0x2024c774e832ac2b059c5bacc3f3736d8e50f5997658daed17ab4bb7dcad5c19,61,0xa478c2975ab1ea89e8196811f51a7b7ade33eb11,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,897587599446926563429,0,0,1894585560969778077,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,473.7646153006043315626955134,NaN
2,2020-11-18 17:28:57+00:00,11283273,0x1c90931720f48b5cd3cea5a1a6c64423dd812a5f3ca79a0875b6b59133708d72,261,0xa478c2975ab1ea89e8196811f51a7b7ade33eb11,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,899033824865797806287,0,0,1897603438171453174,0x1107b6081231d7f256269ad014bf92e041cb08df,473.7732904468778011791623465,NaN
3,2020-11-18 17:28:58+00:00,11283274,0x5a89cc2850fabbda425bd025966be5d470ed1cdb3f4a56bbb34761df53f16d0b,177,0xa478c2975ab1ea89e8196811f51a7b7ade33eb11,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,0,1000000000000000000,470936972491528376645,0,0x427e5697747b64097864bc681d59ae4aab3dccb6,NaN,0.002123426399735452643146684404
4,2020-11-18 17:29:08+00:00,11283275,0x729e1164d74690afc651a874f5d368152bd756522200e69d7f985becd9af0e83,140,0xa478c2975ab1ea89e8196811f51a7b7ade33eb11,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,0,226853230566684541,106832942022602339465,0,0xf00e80f0de9aea0b33aa229a4014572777e422ee,NaN,0.002123438953115134215329270977
...,...,...,...,...,...,...,...,...,...,...,...,...,...
606295,2021-05-16 23:54:16+00:00,12448579,0xde509ce17492cfc39df44b6eeb365d62ff12ae68bbda0aea8302442e100f6d59,19,0xa478c2975ab1ea89e8196811f51a7b7ade33eb11,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,62441405794004325767189,0,0,17419120872776244836,0x80b4d4e9d88d9f78198c56c5a27f3bacb9a685c5,3584.647368260236833764821748,NaN
606299,2021-05-16 23:55:41+00:00,12448585,0x492942e2d0fe5326efbe16538f152212298322293a46615cb0c4ed60354513c2,266,0xa478c2975ab1ea89e8196811f51a7b7ade33eb11,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,5382625218363167550800,0,0,1498976690918701181,0x3328f5f2cecaf00a2443082b657cedeaf70bfaef,3590.866523124008175999461259,NaN
606303,2021-05-16 23:58:21+00:00,12448596,0xf7cc0d1a63f151549aa6686a69ff537b803d59c01673b95beda123b6f895402d,107,0xa478c2975ab1ea89e8196811f51a7b7ade33eb11,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,1579782139168819659205,0,0,439592673942465523,0x01962144d41415cca072900fe87bbe2992a99f10,3593.740826025648594189114445,NaN
606304,2021-05-16 23:58:38+00:00,12448598,0x649b9d25ebe70d0d990b7d8af0dc79159ac43f059bc5eff9270d3e862cdacb18,413,0xa478c2975ab1ea89e8196811f51a7b7ade33eb11,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,500000000000000000000,0,0,139126988746656575,0xb5bd39fc0c57d902271ba6c5cd89c1510a37c0dd,3593.839013582587377059056503,NaN
